In [14]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import SequentialChain, LLMChain

load_dotenv()


True

In [1]:
import neo4j

host = "localhost"
username = "neo4j"
password = "1234qwer"

driver = neo4j.GraphDatabase.driver(f'bolt://localhost:7687', auth=(username, password))
session = driver.session()

def create_graph(query):
    #query = "CREATE   (u:User {id: 'e7779884-ecc5-459e-aae2-41dae12f4454', name: 'Users'}),  (p1:Platform {id: 'e7779884-ecc5-459e-aae2-41dae12f4454', name: 'PayPal'}),  (p2:Platform {id: 'e7779884-ecc5-459e-aae2-41dae12f4454', name: 'eBay'}),  (c:Core {id: 'e7779884-ecc5-459e-aae2-41dae12f4454', name: 'payment'}),  (v:Verb {id: 'e7779884-ecc5-459e-aae2-41dae12f4454', name: 'send'}),  (c2:Core {id: 'e7779884-ecc5-459e-aae2-41dae12f4454', name: 'lump sums'}),  (c3:Core {id: 'e7779884-ecc5-459e-aae2-41dae12f4454', name: 'bank account'}),    (u)-[:USES]->(p1),  (p1)-[:PROVIDES]->(c),  (p2)-[:PLATFORM_FOR]->(c),  (u)-[:RECEIVES]->(c),  (p1)-[:SEND]->(v),  (v)-[:TRANSFERS]->(c2),  (p1)-[:DEPOSITS_INTO]->(c3)"
    session.run(query)

In [16]:
import pandas as pd
import uuid

df = pd.read_csv('odoo_pos.csv')
df.head()

uuids = []
for _ in range(len(df)):
    uuids.append(str(uuid.uuid4()))

def append_feature(str1, str2):
    if(str2 == True):
        return str1 + ' ' + str2
    else:
        return str1

df['id'] = uuids
df['app'] = 'Odoo POS'
df['feature'] = df.apply(lambda row: append_feature(row['Fitur'], row['Subfitur']), axis=1)
df.head()

,Fitur,Subfitur,Sentence,id,app,feature
0,Point of sales,NaN,"With Odoo Point of Sale, run your shops and re...",6ef23729-dd77-49bb-8a3f-246e33bc8d2c,Odoo POS,Point of sales
1,Multi-employee management,NaN,Odoo Point of Sale allows you to manage access...,347279e0-896a-4a23-845e-06029b72274b,Odoo POS,Multi-employee management
2,Product combos,NaN,The Product Combos feature allows users to def...,abd07d20-4a7e-4325-b821-12156052c8d5,Odoo POS,Product combos
3,Product combos,NaN,"In the context of a restaurant, the feature en...",1df2ac61-5d92-4778-9701-c7b66ccef5e9,Odoo POS,Product combos
4,Product combos,NaN,"In retail, this feature allows you to create a...",90fbb98b-ff3b-4b35-906d-36ff94f61afa,Odoo POS,Product combos


In [22]:
df.to_csv('odoo_pos_normalized.csv')

In [17]:
template1 = """You are expert software analyst, you can easily split and classify entity from sentence 
- Categorized as "Core" if the entity is a specific feature of software requirement domain, example (system, application, payment, login) 
- Categorized as "User" if the entity is a specific user of the software, example (Admin, developer, Student, customer, buyer, cashier) 
- Categorized as "Hardware" if the entity is a component of computer hardware, example (Keyboard, mouse, CPU) 
- Categorized as "Platform" if the entity is a third party application or software platform, example (Linux, Paypal, eBay, Apache) 
- Categorized as "Verb" if the entity is miscellaneous actions or verb related to software, example (upgrade, view, checkout, buy, download, delete) 

Given sentence : {sentence}

Output must separated by comma like <entity>,<category>

"""
template2 = """From the output of
<entity>,<category>
{extraction}

and in context sentence : 
{sentence}

- Make connections between entities
- Node label is <category> consists of id the value: {id} and name: <entity>
- Generate a Neo4j cypher query CREATE statement
- Don't display anything in the output other than the Cypher query
"""


prompt1 = ChatPromptTemplate.from_template(template1)
prompt2 = ChatPromptTemplate.from_template(template2)

model = ChatOpenAI(model="gpt-4o-mini", api_key=os.getenv('OPEN_API_KEY'), temperature=0)

chain1 = prompt1 | model
chain2 = prompt2 | model

chain_one = LLMChain(llm=model, prompt=prompt1, output_key="extraction")
chain_two = LLMChain(llm=model, prompt=prompt2, output_key="cypher")

overal_chain = SequentialChain(
    chains=[chain_one, chain_two],
    input_variables=["sentence", "id"],
    output_variables=["extraction", "cypher"]
)

/Users/fahmisyaifudin/miniconda3/envs/llm/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [26]:
for index, row in df.iterrows():
    sentence = row['Sentence']
    sentence_id = row['id']
    message = overal_chain({"sentence": sentence, "id": sentence_id})
    raw_cypher = message['cypher']
    cypher = raw_cypher.replace("\n", "").replace("```cypher", "").replace("```", "")
    create_graph(cypher)

In [6]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained("bert-base-uncased")

/Users/fahmisyaifudin/miniconda3/envs/llm/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
def bert_embedding(text):
    encoded_input = tokenizer(text, return_tensors='pt')
    output = model(**encoded_input)
    pooler_output = output['pooler_output'][0]
    return pooler_output.detach().numpy()

In [48]:
for index, row in df.iterrows():
    records, summary, key = driver.execute_query("MATCH (n {id: $id}) WHERE n.embedding is NULL RETURN DISTINCT n.name", id=row['id'])
    for record in records:
        embedding = bert_embedding(record['n.name'])
        session.run("MATCH (n {id: $id, name: $name}) CALL db.create.setNodeVectorProperty(n, 'embedding', $embedding) SET n.app = $app, n.feature = $feature", id=row['id'], name=record['n.name'], embedding=embedding, app=row['app'], feature=row['feature'])

In [126]:
# session.run("""
# CREATE VECTOR INDEX core_embedding IF NOT EXISTS
# FOR (n:Core)
# ON n.embedding
# OPTIONS {indexConfig: {
#  `vector.dimensions`: 768,
#  `vector.similarity_function`: 'cosine'
# }}""")

# session.run("""
# CREATE VECTOR INDEX platform_embedding IF NOT EXISTS
# FOR (n:Platform)
# ON n.embedding
# OPTIONS {indexConfig: {
#  `vector.dimensions`: 768,
#  `vector.similarity_function`: 'cosine'
# }}""")

# session.run("""
# CREATE VECTOR INDEX user_embedding IF NOT EXISTS
# FOR (n:User)
# ON n.embedding
# OPTIONS {indexConfig: {
#  `vector.dimensions`: 768,
#  `vector.similarity_function`: 'cosine'
# }}""")

# session.run("""
# CREATE VECTOR INDEX hardware_embedding IF NOT EXISTS
# FOR (n:Hardware)
# ON n.embedding
# OPTIONS {indexConfig: {
#  `vector.dimensions`: 768,
#  `vector.similarity_function`: 'cosine'
# }}""")

# session.run("""
# CREATE VECTOR INDEX verb_embedding IF NOT EXISTS
# FOR (n:Verb)
# ON n.embedding
# OPTIONS {indexConfig: {
#  `vector.dimensions`: 768,
#  `vector.similarity_function`: 'cosine'
# }}""")

In [11]:
const_app = 'Odoo POS'
records, summary, key = driver.execute_query("MATCH (n {app: $app})-[r]-(k) return distinct type(r) as relation", app=const_app)
for record in records[:1]:
    relation = record['relation'].lower().replace('_', ' ')
    embedding = bert_embedding(relation)
    session.run(f"MATCH (n)-[r:{record['relation']}]-(k) WHERE n.app = $app CALL db.create.setRelationshipVectorProperty(r, 'embedding', $embedding)", app=const_app, embedding=embedding)


CONSOLIDATES


In [ ]:
# session.run("""
# CREATE VECTOR INDEX core_embedding IF NOT EXISTS
# FOR (n:Core)
# ON n.embedding
# OPTIONS {indexConfig: {
#  `vector.dimensions`: 768,
#  `vector.similarity_function`: 'cosine'
# }}""")

# session.run("""
# CREATE VECTOR INDEX platform_embedding IF NOT EXISTS
# FOR (n:Platform)
# ON n.embedding
# OPTIONS {indexConfig: {
#  `vector.dimensions`: 768,
#  `vector.similarity_function`: 'cosine'
# }}""")

# session.run("""
# CREATE VECTOR INDEX user_embedding IF NOT EXISTS
# FOR (n:User)
# ON n.embedding
# OPTIONS {indexConfig: {
#  `vector.dimensions`: 768,
#  `vector.similarity_function`: 'cosine'
# }}""")

# session.run("""
# CREATE VECTOR INDEX hardware_embedding IF NOT EXISTS
# FOR (n:Hardware)
# ON n.embedding
# OPTIONS {indexConfig: {
#  `vector.dimensions`: 768,
#  `vector.similarity_function`: 'cosine'
# }}""")

# session.run("""
# CREATE VECTOR INDEX verb_embedding IF NOT EXISTS
# FOR (n:Verb)
# ON n.embedding
# OPTIONS {indexConfig: {
#  `vector.dimensions`: 768,
#  `vector.similarity_function`: 'cosine'
# }}""")
